# ReadtheDocs Retrieval Augmented Generation (RAG) using Zilliz Free Tier

In this notebook, we are going to use Milvus documentation pages to create a chatbot about our product.  The chatbot is going to follow RAG steps to retrieve chunks of data using Semantic Vector Search, then the Question + Context will be fed as a Prompt to a LLM to generate an answer.

Many RAG demos use OpenAI for the Embedding Model and ChatGPT for the Generative AI model.  **In this notebook, we will demo a fully open source RAG stack.**

Using open-source Q&A with retrieval saves money since we make free calls to our own data almost all the time - retrieval, evaluation, and development iterations.  We only make a paid call to OpenAI once for the final chat generation step. 

<div>
<img src="../../images/rag_image.png" width="80%"/>
</div>

Let's get started!

In [1]:
# For colab install these libraries in this order:
# !pip install pymilvus, langchain, torch, transformers, python-dotenv, accelerate

# Import common libraries.
import time
import numpy as np

## Download Milvus documentation to a local directory.

The data we’ll use is our own product documentation web pages.  ReadTheDocs is an open-source free software documentation hosting platform, where documentation is written with the Sphinx document generator.

The code block below downloads the web pages into a local directory called `rtdocs`.  

I've already uploaded the `rtdocs` data folder to github, so you should see it if you cloned my repo.

In [2]:
# # Uncomment to download readthedocs pages locally.

# DOCS_PAGE="https://pymilvus.readthedocs.io/en/latest/"
# !echo $DOCS_PAGE

# # Specify encoding to handle non-unicode characters in documentation.
# !wget -r -A.html -P rtdocs --header="Accept-Charset: UTF-8" $DOCS_PAGE

## Start up a Zilliz free tier cluster.

Code in this notebook uses fully-managed Milvus on [Ziliz Cloud free trial](https://cloud.zilliz.com/login).  Choose the default "Starter" option when you provision > Create collection > Give it a name > Create cluster and collection.

💡 Note: To keep your tokens private, best practice is to use an **env variable**.  See [how to save api key in env variable](https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety). <br>

In Jupyter, you also need a .env file (in same dir as notebooks) containing lines like this:
- VARIABLE_NAME=value


In [3]:
# !pip install pymilvus #python sdk for milvus
from pymilvus import connections, utility

import os
from dotenv import load_dotenv
load_dotenv()
TOKEN = os.getenv("ZILLIZ_API_KEY")

# Connect to Zilliz cloud using enpoint URI and API key TOKEN.
# TODO change this before checking into github.
CLUSTER_ENDPOINT="https://in03-xxxx.api.gcp-us-west1.zillizcloud.com:443"
connections.connect(
  alias='default',
  #  Public endpoint obtained from Zilliz Cloud
  uri=CLUSTER_ENDPOINT,
  # API key or a colon-separated cluster username and password
  token=TOKEN,
)

# Check if the server is ready and get colleciton name.
print(f"Type of server: {utility.get_server_version()}")

Type of server: zilliz_cloud


## Load the Embedding Model checkpoint and use it to create vector embeddings
**Embedding model:**  We will use the open-source [sentence transformers](https://www.sbert.net/docs/pretrained_models.html) available on HuggingFace to encode the documentation text.  We will download the model from HuggingFace and run it locally. 

Two model parameters of note below:
1. EMBEDDING_LENGTH refers to the dimensionality or length of the embedding vector. In this case, the embeddings generated for EACH token in the input text will have the SAME length = 768. This size of embedding is often associated with BERT-based models, where the embeddings are used for downstream tasks such as classification, question answering, or text generation. <br><br>
2. MAX_SEQ_LENGTH is the maximum length the encoder model can handle for input sequences. In this case, if sequences longer than 512 tokens are given to the model, everything longer will be (silently!) chopped off.  This is the reason why a chunking strategy is needed to segment input texts into chunks with lengths that will fit in the model's input.

In [4]:
# Import torch.
import torch
from torch.nn import functional as F
from sentence_transformers import SentenceTransformer

# Initialize torch settings
torch.backends.cudnn.deterministic = True
DEVICE = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
print(f"device: {DEVICE}")

# Load the model from huggingface model hub.
model_name = "BAAI/bge-base-en-v1.5"
encoder = SentenceTransformer(model_name, device=DEVICE)
print(type(encoder))
print(encoder)

# Get the model parameters and save for later.
MAX_SEQ_LENGTH = encoder.get_max_seq_length() 
HF_EOS_TOKEN_LENGTH = 1
EMBEDDING_LENGTH = encoder.get_sentence_embedding_dimension()

# Inspect model parameters.
print(f"model_name: {model_name}")
print(f"EMBEDDING_LENGTH: {EMBEDDING_LENGTH}")
print(f"MAX_SEQ_LENGTH: {MAX_SEQ_LENGTH}")

device: cpu
<class 'sentence_transformers.SentenceTransformer.SentenceTransformer'>
SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)
model_name: BAAI/bge-base-en-v1.5
EMBEDDING_LENGTH: 768
MAX_SEQ_LENGTH: 512


## Create a Milvus collection

You can think of a collection in Milvus like a "table" in SQL databases.  The **collection** will contain the 
- **Schema** (or no-schema Milvus Client).  
💡 You'll need the vector `EMBEDDING_LENGTH` parameter from your embedding model.
Typical values are:
   - 768 for sbert embedding models
   - 1536 for ada-002 OpenAI embedding models
- **Vector index** for efficient vector search
- **Vector distance metric** for measuring nearest neighbor vectors
- **Consistency level**
In Milvus, transactional consistency is possible; however, according to the [CAP theorem](https://en.wikipedia.org/wiki/CAP_theorem), some latency must be sacrificed. 💡 Searching movie reviews is not mission-critical, so [`eventually`](https://milvus.io/docs/consistency.md) consistent is fine here.

Some supported [data types](https://milvus.io/docs/schema.md) for Milvus schemas are:
- INT64 - primary key
- FLOAT_VECTOR - embedings = list of `numpy.ndarray` of `numpy.float32` numbers

In [5]:
from pymilvus import (
    FieldSchema, DataType, 
    CollectionSchema, Collection)

# 1. Name your collection.
COLLECTION_NAME = "MilvusDocs"

# 2. Use embedding length from the embedding model.
print(f"Embedding length: {EMBEDDING_LENGTH}")

# 3. Define a minimum expandable schema.
fields = [
    FieldSchema("pk", DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema("vector", DataType.FLOAT_VECTOR, dim=EMBEDDING_LENGTH),
]
schema = CollectionSchema(
		fields,
		description="The schema for docs pages",
		enable_dynamic_field=True
)

# 4. Check if collection already exists, if so drop it.
has = utility.has_collection(COLLECTION_NAME)
if has:
    drop_result = utility.drop_collection(COLLECTION_NAME)
    print(f"Successfully dropped collection: `{COLLECTION_NAME}`")

# 5. Create the collection.
mc = Collection(COLLECTION_NAME, schema, consistency_level="Eventually")

print(f"Successfully created collection: `{COLLECTION_NAME}`")
print(f"Schema: {mc.schema}")

Embedding length: 768
Successfully created collection: `MilvusDocs`
Schema: {'auto_id': True, 'description': 'The schema for docs pages', 'fields': [{'name': 'pk', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 768}}], 'enable_dynamic_field': True}


## Add a Vector Index

The vector index determines the vector **search algorithm** used to find the closest vectors in your data to the query a user submits.  

Most vector indexes use different sets of parameters depending on whether the database is:
- **inserting vectors** (creation mode) - vs - 
- **searching vectors** (search mode) 

Scroll down the [docs page](https://milvus.io/docs/index.md) to see a table listing different vector indexes available on Milvus.  For example:
- FLAT - deterministic exhaustive search
- IVF_FLAT or IVF_SQ8 - Hash index (stochastic approximate search)
- HNSW - Graph index (stochastic approximate search)
- AUTOINDEX - Automatically determined based on OSS vs [Zilliz cloud](https://docs.zilliz.com/docs/autoindex-explained), type of GPU, size of data.

Besides a search algorithm, we also need to specify a **distance metric**, that is, a definition of what is considered "close" in vector space.  In the cell below, the [`HNSW`](https://github.com/nmslib/hnswlib/blob/master/ALGO_PARAMS.md) search index is chosen (Milvus OSS default AUTOINDEX).  Its possible distance metrics are one of:
- L2 - L2-norm
- IP - Dot-product
- COSINE - Angular distance

💡 Most use cases work better with normalized embeddings, in which case L2 is useless (every vector has length=1) and IP and COSINE are the same.  Only choose L2 if you plan to keep your embeddings unnormalized.

In [6]:
# 5. Drop the index, in case it already exists.
mc.drop_index()

# 6. Add a default search index to the collection.
index_params = {
    "index_type": "AUTOINDEX",
    "metric_type": "COSINE", 
    # No params for AUTOINDEX
    # "params": {}
    }

# 7. Specify column name which contains the vector.
mc.create_index(
    field_name="vector", 
    index_params=index_params)

# Get loading progress
mc.load()
progress = utility.loading_progress(COLLECTION_NAME)
print(progress)

{'loading_progress': '100%'}


In [7]:
## Read docs into LangChain
#!pip install langchain 
from langchain.document_loaders import ReadTheDocsLoader

loader = ReadTheDocsLoader("rtdocs/pymilvus.readthedocs.io/en/latest/"
                           , encoding="utf-8"
                           , features="html.parser")
docs = loader.load()

num_documents = len(docs)
print(f"loaded {num_documents} documents")

loaded 8 documents


## Chunking

Before embedding, it is necessary to decide your chunk strategy, chunk size, and chunk overlap.  In this demo, I will use:
- **Strategy** = Use markdown header hierarchies.  Keep markdown sections together unless they are too long.
- **Chunk size** = Use the embedding model's parameter `MAX_SEQ_LENGTH`
- **Overlap** = Rule-of-thumb 10-15%
- **Function** = 
  - Langchain's `HTMLHeaderTextSplitter` to split markdown sections.
  - Langchain's `RecursiveCharacterTextSplitter` to split up long reviews recursively.


Notice below, each chunk is grounded with the document source page.  <br>
In addition, header titles are kept together with the chunk of markdown text.

In [8]:
from langchain.text_splitter import HTMLHeaderTextSplitter, RecursiveCharacterTextSplitter
from bs4 import BeautifulSoup

# Define the headers to split on for the HTMLHeaderTextSplitter
headers_to_split_on = [
    ("h1", "Header 1"),
    ("h2", "Header 2"),
]
# Create an instance of the HTMLHeaderTextSplitter
html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

# Use the embedding model parameters.
chunk_size = MAX_SEQ_LENGTH - HF_EOS_TOKEN_LENGTH
chunk_overlap = np.round(chunk_size * 0.10, 0)

# Create an instance of the RecursiveCharacterTextSplitter
child_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap,
    length_function = len,
)

# Split the HTML text using the HTMLHeaderTextSplitter.
start_time = time.time()
html_header_splits = []
for doc in docs:
    soup = BeautifulSoup(doc.page_content, 'html.parser')
    splits = html_splitter.split_text(str(soup))
    for split in splits:
        # Add the source URL and header values to the metadata
        metadata = {}
        new_text = split.page_content
        for header_name, metadata_header_name in headers_to_split_on:
            header_value = new_text.split("¶ ")[0].strip()
            metadata[header_name] = header_value
            try:
                new_text = new_text.split("¶ ")[1].strip()
            except:
                break
        split.metadata = {
            **metadata,
            "source": doc.metadata["source"]
        }
        # Add the header to the text
        split.page_content = split.page_content
    html_header_splits.extend(splits)

# Split the documents further into smaller, recursive chunks.
chunks = child_splitter.split_documents(html_header_splits)

end_time = time.time()
print(f"chunking time: {end_time - start_time}")
print(f"docs: {len(docs)}, split into: {len(html_header_splits)}")
print(f"split into chunks: {len(chunks)}, type: list of {type(chunks[0])}") 

# Inspect a chunk.
print()
print("Looking at a sample chunk...")
print(chunks[0].page_content[:100])
print(chunks[0].metadata)

# # TODO - remove this before saving in github.
# # Print the child splits with their associated header metadata
# print()
# for child in chunks:
#     print(f"Content: {child.page_content}")
#     print(f"Metadata: {child.metadata}")
#     print()

chunking time: 0.012528181076049805
docs: 8, split into: 8
split into chunks: 156, type: list of <class 'langchain.schema.document.Document'>

Looking at a sample chunk...
Installation¶ Installing via pip¶ PyMilvus is in the Python Package Index. PyMilvus only support pyt
{'h1': 'Installation', 'h2': 'Installing via pip', 'source': 'rtdocs/pymilvus.readthedocs.io/en/latest/install.html'}


In [9]:
# Clean up the metadata urls
for doc in chunks:
    new_url = doc.metadata["source"]
    new_url = new_url.replace("rtdocs", "https:/")
    doc.metadata.update({"source": new_url})

print(chunks[0].page_content[:100])
print(chunks[0].metadata)

Installation¶ Installing via pip¶ PyMilvus is in the Python Package Index. PyMilvus only support pyt
{'h1': 'Installation', 'h2': 'Installing via pip', 'source': 'https://pymilvus.readthedocs.io/en/latest/install.html'}


## Insert data into Milvus

For each original text chunk, we'll write the quadruplet (`vector, text, source, h1, h2`) into the database.

<div>
<img src="../../images/db_insert.png" width="80%"/>
</div>

Milvus and Milvus Lite support loading data from:
- pandas dataframes 
- list of dictionaries

Below, we use the embedding model provided by HuggingFace, download its checkpoint, and run it locally as the encoder.  

In [10]:
# Convert chunks to a list of dictionaries.
chunk_list = []
for chunk in chunks:

    # Generate embeddings using encoder from HuggingFace.
    embeddings = torch.tensor(encoder.encode([chunk.page_content]))
    embeddings = F.normalize(embeddings, p=2, dim=1)
    converted_values = list(map(np.float32, embeddings))[0]
    
    # Only use h1, h2. Truncate the metadata in case too long.
    try:
        h2 = chunk.metadata['h2'][:50]
    except:
        h2 = ""
    # Assemble embedding vector, original text chunk, metadata.
    chunk_dict = {
        'vector': converted_values,
        'text': chunk.page_content,
        'source': chunk.metadata['source'],
        'h1': chunk.metadata['h1'][:50],
        'h2': h2,
    }
    chunk_list.append(chunk_dict)

# # TODO - remove this before saving in github.
# for chunk in chunk_list[:1]:
#     print(chunk)

In [11]:
# Insert data into the Milvus collection.

print("Start inserting entities")
start_time = time.time()
insert_result = mc.insert(chunk_list)

end_time = time.time()
print(f"Milvus insert time for {len(chunk_list)} vectors: {end_time - start_time} seconds")

# After final entity is inserted, call flush to stop growing segments left in memory.
mc.flush() 

# Inspect results.
print(mc.partitions) # list[Partition] objects


Start inserting entities
Milvus insert time for 156 vectors: 0.3762798309326172 seconds
[{"name":"_default","collection_name":"MilvusDocs","description":""}]


## Ask a question about your data

So far in this demo notebook: 
1. Your custom data has been mapped into a vector embedding space
2. Those vector embeddings have been saved into a vector database

Next, you can ask a question about your custom data!

💡 In LLM vocabulary:
> **Query** is the generic term for user questions.  
A query is a list of multiple individual questions, up to maybe 1000 different questions!

> **Question** usually refers to a single user question.  
In our example below, the user question is "What is AUTOINDEX in Milvus Client?"

> **Semantic Search** = search all the documentation embeddings to find the `TOP_K` documentation chunks with the closest embeddings to the user's query.

💡 The same model should always be used for consistency for all the embeddings data and the query.

In [12]:
# Define a sample question about your data.
QUESTION1 = "What do the parameters for HNSW mean?"
QUESTION2 = "What are good default values for HNSW parameters with 25K vectors dim 768?"
QUESTION3 = "Default distance metric used in AUTOINDEX?"
QUERY = [QUESTION1, QUESTION2, QUESTION3]

# Inspect the length of the query.
QUERY_LENGTH = len(QUESTION2)
print(f"query length: {QUERY_LENGTH}")

query length: 74


In [13]:
# SELECT A PARTICULAR QUESTION TO ASK.

SAMPLE_QUESTION = QUESTION1

## Execute a vector search

Search Milvus using [PyMilvus API](https://milvus.io/docs/search.md).

💡 By their nature, vector searches are "semantic" searches.  For example, if you were to search for "leaky faucet": 
> **Traditional Key-word Search** - either or both words "leaky", "faucet" would have to match some text in order to return a web page or link text to the document.

> **Semantic search** - results containing words "drippy" "taps" would be returned as well because these words mean the same thing even though they are different words,

In [14]:
# RETRIEVAL USING MILVUS API.

# Before conducting a search based on a query, you need to load the data into memory.
mc.load()
print("Loaded milvus collection into memory.")

# Embed the question using the same encoder.
embedded_question = torch.tensor(encoder.encode([SAMPLE_QUESTION]))
# Normalize embeddings to unit length.
embedded_question = F.normalize(embedded_question, p=2, dim=1)
# Convert the embeddings to list of list of np.float32.
embedded_question = list(map(np.float32, embedded_question))

# Return top k results with AUTOINDEX.
TOP_K = 3

# Run semantic vector search using your query and the vector database.
start_time = time.time()
results = mc.search(
    data=embedded_question, 
    anns_field="vector", 
    # No params for AUTOINDEX
    param={},
    # Milvus can utilize metadata to enhance the search experience in boolean expressions.
    # expr="",
    output_fields=["h1", "h2", "text", "source"], 
    limit=TOP_K,
    consistency_level="Eventually"
    )

elapsed_time = time.time() - start_time
print(f"Milvus search time: {elapsed_time} sec")

# Inspect search result.
print(f"type: {type(results)}, count: {len(results[0])}")


Loaded milvus collection into memory.
Milvus search time: 0.05107903480529785 sec
type: <class 'pymilvus.client.abstract.SearchResult'>, count: 3


## Assemble and inspect the search result

The search result is in the variable `results[0]` of type `'pymilvus.orm.search.SearchResult'`.  

In [15]:
def assemble_retrieved_context(retrieved_results, num_shot_answers=3):
    
    # Assemble the context as a stuffed string.
    context = ""
    i = 1
    for r in retrieved_results[0]:
        text = r.entity.text
        if i <= num_shot_answers:  # only first n results
            context += f"{text} "
        i += 1
    print(f"Length of context: {len(context)}")

    # Also save the context metadata to retrieve along with the answer.
    context_metadata = []
    i = 1
    for r in retrieved_results[0]:
        if i <= num_shot_answers:
            context_metadata.append({
                "h1": r.entity.h1,
                "h2": r.entity.h2,
                "source": r.entity.source,
            })
        i += 1

    return context, context_metadata

In [16]:
# # TODO - remove printing before saving in github.
# for n, hits in enumerate(results):
#     print(f"{n}th query result")
#     for hit in hits:
#         print(hit)

# Assemble the context and context metadata.
context, context_metadata = assemble_retrieved_context(results, num_shot_answers=TOP_K)
print(len(context_metadata), len(context_metadata))

Length of context: 1519
3 3


## Use an LLM to Generate a chat response to the user's question using the Retrieved Context.

Below, we'll use an open, very tiny generative AI model, or LLM, available on HuggingFace.  Many demos use OpenAI as the LLM choice instead.

In [17]:
def assemble_answer_sources(answer, context_metadata):
    """Assemble the answer and grounding sources into a string"""
    grounded_answer = f"Answer: {answer}\n"
    grounded_answer += "Grounding sources and citations:\n"

    for metadata in context_metadata:
        try:
            grounded_answer += f"'h1': {metadata['h1']}, 'h2':{metadata['h2']}\n"
        except:
            pass
        try:
            grounded_answer += f"'source': {metadata['source']}"
        except:
            pass
        
    return grounded_answer

In [18]:
# BASELINING THE LLM: ASK A QUESTION WITHOUT ANY RETRIEVED CONTEXT.

from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

# Load the Hugging Face auto-regressive LLM checkpoint.
tiny_llm = "deepset/tinyroberta-squad2"
tokenizer = AutoTokenizer.from_pretrained(tiny_llm)

# context cannot be empty so just put random text in it.
QA_input = {
    'question': SAMPLE_QUESTION,
    'context': 'The quick brown fox jumped over the lazy dog'
}

nlp = pipeline('question-answering', 
               model=tiny_llm, 
               tokenizer=tokenizer)
result = nlp(QA_input)

# Print the question and answer.
print(f"Question: {SAMPLE_QUESTION}")
print(f"Answer: {result['answer']}")

# The baseline LLM chat is not very helpful.

Question: What do the parameters for HNSW mean?
Answer: lazy dog


In [19]:
# NOW ASK THE SAME LLM THE SAME QUESTION USING THE RETRIEVED CONTEXT.
QA_input = {
    'question': SAMPLE_QUESTION,
    'context': context,
}

nlp = pipeline('question-answering', 
               model=tiny_llm, 
               tokenizer=tokenizer)
result = nlp(QA_input)

# Print the question and answer along with grounding sources and citations.
answer = assemble_answer_sources(result['answer'], context_metadata)
print(f"Question: {SAMPLE_QUESTION}")
print(answer)

# That answer looks a little better!

Question: What do the parameters for HNSW mean?
Answer: M: Maximum degree of the node
Grounding sources and citations:
'h1': Index, 'h2':Milvus support to create index to accelerate vecto
'source': https://pymilvus.readthedocs.io/en/latest/param.html'h1': Source code for milvus.client.types from enum impo, 'h2':
'source': https://pymilvus.readthedocs.io/en/latest/_modules/milvus/client/types.html'h1': Index, 'h2':Milvus support to create index to accelerate vecto
'source': https://pymilvus.readthedocs.io/en/latest/param.html


## Use OpenAI to generate a more human-like chat response to the user's question 

We've practiced retrieval for free on our own data using open-source LLMs.  <br>

Now let's make a call to the paid OpenAI GPT.

💡 Note: We’re using a temperature of 0.0 to enable reproducible experiments. For use cases that need to always be factually grounded, use very low temperature values while more creative tasks can benefit from higher temperatures.

In [20]:
import openai, pprint
from openai import OpenAI
# from dotenv import load_dotenv, find_dotenv

# Define the generation llm model to use.
LLM_NAME = "gpt-3.5-turbo-1106"

# See how to save api key in env variable.
# https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety
openai_client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [21]:
def prepare_response(response):
    return response["choices"][-1]["message"]["content"]

def generate_response(
    llm, temperature=0.0, 
    grounding_sources=None,
    system_content="", assistant_content="", user_content=""):
    """Generate response from an LLM."""

    try:
        response = openai.ChatCompletion.create(
            model=llm,
            temperature=temperature,
            api_key=openai.api_key,
            messages=[
                {"role": "system", "content": system_content},
                {"role": "assistant", "content": assistant_content},
                {"role": "user", "content": user_content},
            ],
        )
        answer = prepare_response(response=response)
    
        # Add the grounding sources and citations.
        answer = assemble_answer_sources(answer, grounding_sources)
        return answer

    except Exception as e:
        print(f"Exception: {e}")
    return ""

In [22]:
# CAREFUL!! THIS COSTS MONEY!!
# Generate response

prompt = f"""Answer the question using the context provided. Be succinct.
Echo in the answer the Grounding Sources.
Grounding sources: {context_metadata}
"""

response = openai_client.chat.completions.create(
    messages=[
                {
            "role": "system",
            "content": prompt,
        },
        {
            "role": "user",
            "content": f"question: {SAMPLE_QUESTION}, context: {context}",
        }
    ],
    model=LLM_NAME,
)

# Print the question and answer along with grounding sources and citations.
print(f"Question: {SAMPLE_QUESTION}")

# Print all choices in the response
for i, choice in enumerate(response.choices, 1):
    # Print the answer
    pprint.pprint(f"Answer: {choice.message.content}")
    print("\n")

# Question1: What do the parameters for HNSW mean?
# Answer: Correct for M, close for efConstruction and ef.
# Best answer:  M: maximum degree of nodes in a layer of the graph. 
# efConstruction: number of nearest neighbors to consider when connecting nodes in the graph.
# ef: number of nearest neighbors to consider when searching for similar vectors. 

# Question2: What are good default values for HNSW parameters with 25K vectors dim 768?
# Answer: M=16, efConstruction=500, and ef=64
# Best answer:  M=16, efConstruction=32, ef=32

# Question3: what is the default distance metric used in AUTOINDEX in Milvus?
# Answer: L2 
# Trick answer:  IP inner product, not yet updated in documentation still says L2.

Question: What do the parameters for HNSW mean?
('Answer: The parameters for HNSW include M, which is the maximum degree of '
 'the node, and efConstruction, which takes effect in the stage of index '
 'construction. The M parameter specifies the maximum degree of nodes on each '
 'layer of the graph, and the efConstruction parameter specifies a search '
 'range during index construction. The ef parameter is used for specifying a '
 'search range when searching targets. These parameters affect the performance '
 'of the HNSW index in Milvus.\n'
 '\n'
 'Sources:\n'
 '- https://pymilvus.readthedocs.io/en/latest/param.html')




## New!! Demo Zilliz Pipelines

Zilliz Pipelines is an easy way to automate data embedding and vector indexing.  It simplifies the process of getting your data into a vector database and querying it!

Available in a Web UI or restful API call.

<div>
<img src="../../images/zilliz_pipeline_s3.png" width="80%"/>
</div>

The basic workflow is:
1. Upload your data to S3 or GCS.
2. Open a browser to `https://cloud.zilliz.com/` and create a Cluster.
3. Navigate through the left-side menu to `Pipelines` and follow guided steps to upload your data.
4. Query in the web console or using API calls (shown below).

In [23]:
import requests, json

# Define the URL, headers, and data
url = "https://controller.api.gcp-us-west1.zillizcloud.com/v1/pipelines/pipe-458714b66ffc8ff3ab1bf1/run"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {TOKEN}",
}
data = {
    "data": {
        "query_text": SAMPLE_QUESTION
    },
    "params": {
        "limit": TOP_K,
        "offset": 0,
        "outputFields": ["chunk_text", "chunk_id", "doc_name", "source"],
        "filter": "chunk_id >= 0"
    }
}

# Send the POST request
response = requests.post(url, headers=headers, json=data)
# print(type(response))
# # Print the response
# pprint.pprint(response.json())

# Assemble context from Pipeline retriever.
pipeline_context = ""
pipeline_context_metadata = []

# Loop through each result in the response
for result in response.json()['data']['result']:
    print(result)
    # Add the chunk_text to the context
    pipeline_context += result['chunk_text'] + " "

    # Add the source to the context metadata
    pipeline_context_metadata.append({
        "source": result['source'],
    })

print(f"Length context: {len(pipeline_context)}, Len metadata: {len(pipeline_context_metadata)}")

{'id': 446011086634220126, 'distance': 0.8051614761352539, 'source': 'https://pymilvus.readthedocs.io/en/latest/api.html', 'doc_name': 'api.html', 'chunk_text': 'IVF SQ8 Hybrid index. See IVF_SQ8H.\n\n`IVF_PQ` _= 6_¶\n\n    \n\nIVF PQ index. See IVF_PQ.\n\n`HNSW` _= 11_¶\n\n    \n\nHNSW index. See HNSW.\n\n`ANNOY` _= 12_¶\n\n    \n\nANNOY index. See ANNOY.\n\n`IVFLAT` _= 2_¶\n\n    \n\nAlternative name for IVF_FLAT. Reserved for compatibility.\n\n`IVF_SQ8_H` _= 5_¶\n\n    \n\nAlternative name for IVF_SQ8H. Reserved for compatibility.\n\n## Metric Type¶\n\n_class_`milvus.``MetricType`[source]¶\n\n    \n\nMetric type enum.\n\n`INVALID` _= 0_¶\n\n    \n\nInvalid metric type.\n\n`L2` _= 1_¶\n\n    \n\nEuclidean distance. A metric for float vectors. See Euclidean distance.\n\n`IP` _= 2_¶\n\n    \n\nInner product. A metric for float vectors. See Inner Product.\n\n`HAMMING` _= 3_¶\n\n    \n\nHamming distance. A metric for binary vectors. See Hamming distance.\n\n`JACCARD` _= 4_¶\n\n    \n\nJa

In [24]:
# CAREFUL!! THIS COSTS MONEY!!
# Generate response
prompt = f"""Answer the question using the context provided. Be succinct.
Echo in the answer the Grounding sources.
Grounding sources: {pipeline_context_metadata}
"""

response_pipeline = openai_client.chat.completions.create(
    messages=[
                {
            "role": "system",
            "content": prompt,
        },
        {
            "role": "user",
            "content": f"question: {SAMPLE_QUESTION}, context: {pipeline_context}",
        }
    ],
    model=LLM_NAME,
)

# Print the question and answer along with grounding sources and citations.
print(f"Question: {SAMPLE_QUESTION}")

# Print all choices in the response
for i, choice in enumerate(response_pipeline.choices, 1):
    pprint.pprint(f"Answer: {choice.message.content}")
    print("\n")

# Question1: What do the parameters for HNSW mean?
# Answer: Only gave parameter 'M'.  Missed 'efConstruction' and 'ef'.
# Best answer:  M: maximum degree of nodes in a layer of the graph. 
# efConstruction: number of nearest neighbors to consider when connecting nodes in the graph.
# ef: number of nearest neighbors to consider when searching for similar vectors. 

# Question2: What are good default values for HNSW parameters with 25K vectors dim 768?
# Answer: `M=16` and `efConstruction=200`
# Best answer:  M=16, efConstruction=32, ef=32

# Question3: what is the default distance metric used in AUTOINDEX in Milvus?
# Answer: L2
# Trick answer:  IP inner product, not yet updated in documentation still says L2.

Question: What do the parameters for HNSW mean?
('Answer: The parameter for HNSW specifies the number of connection edges on '
 'each subgraph node. By setting the parameter for HNSW, you determine the '
 'trade-off between recall and index build time. You can find more information '
 'on index parameters in the PyMilvus documentation.\n'
 '\n'
 'Sources:\n'
 '- [PyMilvus Documentation - '
 'API](https://pymilvus.readthedocs.io/en/latest/api.html)\n'
 '- [PyMilvus Documentation - '
 'Tutorial](https://pymilvus.readthedocs.io/en/latest/tutorial.html)\n'
 '- [PyMilvus Documentation - '
 'Stub](https://pymilvus.readthedocs.io/en/latest/_modules/milvus/client/stub.html)')




In [25]:
# Drop collection
utility.drop_collection(COLLECTION_NAME)

In [26]:
# Props to Sebastian Raschka for this handy watermark.
# !pip install watermark

%load_ext watermark
%watermark -a 'Christy Bergman' -v -p torch,transformers,sentence_transformers,pymilvus,langchain,openai --conda

Author: Christy Bergman

Python implementation: CPython
Python version       : 3.11.6
IPython version      : 8.18.1

torch                : 2.1.1
transformers         : 4.35.2
sentence_transformers: 2.2.2
pymilvus             : 2.3.4
langchain            : 0.0.322
openai               : 1.3.7

conda environment: py311

